In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
import joblib
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("FINSIGHT: CALENDAR EVENT EXPENSE PREDICTION & SAVINGS RECOMMENDATION")
print("="*70)

# ============================================================================
# MODEL 1: EVENT EXPENSE PREDICTOR
# ============================================================================
print("\n" + "="*70)
print("MODEL 1: EVENT EXPENSE PREDICTOR")
print("="*70)

print("\n📂 Loading Datasets...")
df_calendar = pd.read_csv('D:/Know I/Final/data/finsight_calendar_events.csv')
df_transactions = pd.read_csv('D:/Know I/Final/data/finsight_transactions.csv')
df_users = pd.read_csv('D:/Know I/Final/data/finsight_users.csv')

print(f"✅ Calendar events: {len(df_calendar)}")
print(f"✅ Transactions: {len(df_transactions)}")
print(f"✅ Users: {len(df_users)}")

# Convert dates
df_calendar['event_date'] = pd.to_datetime(df_calendar['event_date'])
df_transactions['date'] = pd.to_datetime(df_transactions['date'])

# Feature Engineering for Event Expense Prediction
print("\n🔧 Engineering features for event expense prediction...")

# For each calendar event, find actual expenses around that date (±7 days)
def get_actual_expense_for_event(row):
    """Get actual expense within 7 days of event"""
    user_id = row['user_id']
    event_date = row['event_date']
    event_type = row['event_type']
    
    # Define category mapping for event types
    category_map = {
        'Bill': ['Bills', 'Utilities', 'Rent', 'Insurance'],
        'Celebration': ['Dining', 'Entertainment', 'Shopping'],
        'Travel': ['Travel', 'Transport'],
        'Healthcare': ['Healthcare'],
        'Maintenance': ['Shopping', 'Transport'],
        'Shopping': ['Shopping'],
        'Education': ['Education'],
        'Financial': ['Bills', 'Insurance']
    }
    
    categories = category_map.get(event_type, ['Dining', 'Entertainment', 'Shopping'])
    
    # Get transactions within ±7 days
    start_date = event_date - pd.Timedelta(days=7)
    end_date = event_date + pd.Timedelta(days=7)
    
    user_txns = df_transactions[
        (df_transactions['user_id'] == user_id) &
        (df_transactions['date'] >= start_date) &
        (df_transactions['date'] <= end_date) &
        (df_transactions['transaction_type'] == 'debit') &
        (df_transactions['category'].isin(categories))
    ]
    
    if len(user_txns) > 0:
        return user_txns['amount'].sum()
    else:
        return row['predicted_expense']  # Use predicted if no actual data

# Add actual expenses
print("   • Calculating actual expenses for events...")
df_calendar['actual_expense'] = df_calendar.apply(get_actual_expense_for_event, axis=1)

# Add user profile features
df_calendar = df_calendar.merge(df_users, on='user_id', how='left')

# Add time-based features
df_calendar['day_of_week'] = df_calendar['event_date'].dt.dayofweek
df_calendar['day_of_month'] = df_calendar['event_date'].dt.day
df_calendar['quarter'] = df_calendar['event_date'].dt.quarter
df_calendar['is_weekend'] = df_calendar['day_of_week'].isin([5, 6]).astype(int)
df_calendar['is_festive_season'] = df_calendar['month'].isin([10, 11, 12, 1]).astype(int)
df_calendar['is_summer'] = df_calendar['month'].isin([5, 6]).astype(int)

# Add historical spending patterns
print("   • Adding historical spending patterns...")
user_monthly_avg = df_transactions[df_transactions['transaction_type'] == 'debit'].groupby('user_id')['amount'].mean().reset_index()
user_monthly_avg.columns = ['user_id', 'user_avg_monthly_expense']
df_calendar = df_calendar.merge(user_monthly_avg, on='user_id', how='left')

# Encode categorical variables
le_user = LabelEncoder()
le_event_name = LabelEncoder()
le_event_type = LabelEncoder()
le_personality = LabelEncoder()
le_risk = LabelEncoder()

df_calendar['user_encoded'] = le_user.fit_transform(df_calendar['user_id'])
df_calendar['event_name_encoded'] = le_event_name.fit_transform(df_calendar['event_name'])
df_calendar['event_type_encoded'] = le_event_type.fit_transform(df_calendar['event_type'])
df_calendar['personality_encoded'] = le_personality.fit_transform(df_calendar['spending_personality'])
df_calendar['risk_encoded'] = le_risk.fit_transform(df_calendar['risk_tolerance'])

# Prepare features and target
feature_cols_model1 = [
    'user_encoded', 'event_type_encoded', 'year', 'month', 'quarter',
    'day_of_week', 'is_weekend', 'is_festive_season', 'is_summer',
    'predicted_expense', 'monthly_salary', 'savings_rate',
    'personality_encoded', 'risk_encoded', 'user_avg_monthly_expense',
    'is_recurring'
]

X_event = df_calendar[feature_cols_model1].copy()
y_event = df_calendar['actual_expense'].copy()

# Convert boolean to int
X_event['is_recurring'] = X_event['is_recurring'].astype(int)

print(f"\n✅ Feature matrix shape: {X_event.shape}")
print(f"✅ Target (actual expense) shape: {y_event.shape}")

# Train-test split
X_train_e, X_test_e, y_train_e, y_test_e = train_test_split(X_event, y_event, test_size=0.2, random_state=42)

print(f"✅ Training samples: {len(X_train_e)}")
print(f"✅ Test samples: {len(X_test_e)}")

# Train models
print("\n🤖 Training models for event expense prediction...")

models_event = {
    'XGBoost': xgb.XGBRegressor(n_estimators=300, learning_rate=0.05, max_depth=6, random_state=42),
    'LightGBM': LGBMRegressor(n_estimators=300, learning_rate=0.05, max_depth=6, random_state=42, verbose=-1),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=250, learning_rate=0.05, max_depth=6, random_state=42),
    'RandomForest': RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42),
}

results_event = {}

for name, model in models_event.items():
    model.fit(X_train_e, y_train_e)
    y_pred_test = model.predict(X_test_e)
    y_pred_test = np.maximum(y_pred_test, 0)
    
    test_mae = mean_absolute_error(y_test_e, y_pred_test)
    test_rmse = np.sqrt(mean_squared_error(y_test_e, y_pred_test))
    test_r2 = r2_score(y_test_e, y_pred_test)
    test_accuracy = np.mean(np.abs((y_test_e - y_pred_test) / (y_test_e + 1)) <= 0.20) * 100
    
    results_event[name] = {
        'model': model,
        'mae': test_mae,
        'rmse': test_rmse,
        'r2': test_r2,
        'accuracy': test_accuracy
    }
    
    print(f"\n{name}:")
    print(f"   MAE: ₹{test_mae:.2f} | RMSE: ₹{test_rmse:.2f} | R²: {test_r2:.4f} | Acc: {test_accuracy:.2f}%")

# Select best model
best_model_name_event = max(results_event, key=lambda x: results_event[x]['accuracy'])
best_model_event = results_event[best_model_name_event]['model']

print(f"\n🏆 Best Model: {best_model_name_event}")
print(f"   Accuracy: {results_event[best_model_name_event]['accuracy']:.2f}%")
print(f"   MAE: ₹{results_event[best_model_name_event]['mae']:.2f}")

# ============================================================================
# MODEL 2: SAVINGS RECOMMENDATION SYSTEM
# ============================================================================
print("\n" + "="*70)
print("MODEL 2: SAVINGS RECOMMENDATION SYSTEM")
print("="*70)

print("\n🔧 Engineering features for savings recommendation...")

# Create training data for savings recommendations
# For each event, calculate:
# 1. Budget shortfall/surplus
# 2. Savings needed
# 3. Recommended actions

savings_data = df_calendar.copy()

# Calculate financial metrics
savings_data['expense_vs_predicted_ratio'] = savings_data['actual_expense'] / (savings_data['predicted_expense'] + 1)
savings_data['monthly_discretionary_income'] = savings_data['monthly_salary'] * (1 - savings_data['savings_rate'])
savings_data['expense_to_income_ratio'] = savings_data['actual_expense'] / savings_data['monthly_salary']

# Calculate savings needed (how much to save before event)
# If actual > predicted, need more savings
savings_data['savings_needed'] = np.maximum(
    savings_data['actual_expense'] - savings_data['predicted_expense'], 
    0
)

# Calculate recommended monthly savings (assuming 3-month preparation)
savings_data['recommended_monthly_saving'] = savings_data['savings_needed'] / 3

# Categorize savings urgency
def categorize_urgency(row):
    ratio = row['expense_to_income_ratio']
    if ratio > 0.5:
        return 'High'
    elif ratio > 0.3:
        return 'Medium'
    else:
        return 'Low'

savings_data['urgency'] = savings_data.apply(categorize_urgency, axis=1)

# Encode urgency
le_urgency = LabelEncoder()
savings_data['urgency_encoded'] = le_urgency.fit_transform(savings_data['urgency'])

# Prepare features for savings prediction
feature_cols_model2 = [
    'user_encoded', 'event_type_encoded', 'year', 'month',
    'predicted_expense', 'actual_expense', 'monthly_salary', 'savings_rate',
    'personality_encoded', 'risk_encoded', 'is_festive_season', 'is_summer',
    'expense_to_income_ratio', 'expense_vs_predicted_ratio'
]

X_savings = savings_data[feature_cols_model2].copy()
y_savings = savings_data['recommended_monthly_saving'].copy()

print(f"\n✅ Feature matrix shape: {X_savings.shape}")
print(f"✅ Target (recommended savings) shape: {y_savings.shape}")

# Train-test split
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_savings, y_savings, test_size=0.2, random_state=42)

print(f"✅ Training samples: {len(X_train_s)}")
print(f"✅ Test samples: {len(X_test_s)}")

# Train models
print("\n🤖 Training models for savings recommendation...")

models_savings = {
    'XGBoost': xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=5, random_state=42),
    'LightGBM': LGBMRegressor(n_estimators=200, learning_rate=0.05, max_depth=5, random_state=42, verbose=-1),
    'RandomForest': RandomForestRegressor(n_estimators=150, max_depth=8, random_state=42),
}

results_savings = {}

for name, model in models_savings.items():
    model.fit(X_train_s, y_train_s)
    y_pred_test = model.predict(X_test_s)
    y_pred_test = np.maximum(y_pred_test, 0)
    
    test_mae = mean_absolute_error(y_test_s, y_pred_test)
    test_rmse = np.sqrt(mean_squared_error(y_test_s, y_pred_test))
    test_r2 = r2_score(y_test_s, y_pred_test)
    
    results_savings[name] = {
        'model': model,
        'mae': test_mae,
        'rmse': test_rmse,
        'r2': test_r2
    }
    
    print(f"\n{name}:")
    print(f"   MAE: ₹{test_mae:.2f} | RMSE: ₹{test_rmse:.2f} | R²: {test_r2:.4f}")

# Select best model
best_model_name_savings = min(results_savings, key=lambda x: results_savings[x]['mae'])
best_model_savings = results_savings[best_model_name_savings]['model']

print(f"\n🏆 Best Model: {best_model_name_savings}")
print(f"   MAE: ₹{results_savings[best_model_name_savings]['mae']:.2f}")
print(f"   RMSE: ₹{results_savings[best_model_name_savings]['rmse']:.2f}")

# Save all models and encoders
print("\n" + "="*70)
print("💾 SAVING MODELS & ENCODERS")
print("="*70)

joblib.dump(best_model_event, 'event_expense_predictor.pkl')
joblib.dump(best_model_savings, 'savings_recommender.pkl')
joblib.dump(le_user, 'le_user.pkl')
joblib.dump(le_event_name, 'le_event_name.pkl')
joblib.dump(le_event_type, 'le_event_type.pkl')
joblib.dump(le_personality, 'le_personality.pkl')
joblib.dump(le_risk, 'le_risk.pkl')
joblib.dump(le_urgency, 'le_urgency.pkl')
joblib.dump(feature_cols_model1, 'feature_cols_event.pkl')
joblib.dump(feature_cols_model2, 'feature_cols_savings.pkl')
joblib.dump(df_users, 'users_reference.pkl')

print("✅ Event expense predictor: event_expense_predictor.pkl")
print("✅ Savings recommender: savings_recommender.pkl")
print("✅ All encoders saved")

# ============================================================================
# INTEGRATED PREDICTION & RECOMMENDATION SYSTEM
# ============================================================================
print("\n" + "="*70)
print("🎯 INTEGRATED SYSTEM: PREDICT & RECOMMEND")
print("="*70)

def predict_and_recommend(user_id, event_name, event_type, year, month, predicted_expense):
    """
    Complete system: Predict actual expense and recommend savings
    
    Parameters:
    -----------
    user_id : str - User ID (e.g., 'U001')
    event_name : str - Event name (e.g., 'Trip to Goa')
    event_type : str - Event type (e.g., 'Travel')
    year : int - Year
    month : int - Month (1-12)
    predicted_expense : float - User's estimated expense
    
    Returns:
    --------
    dict - Complete analysis with predictions and recommendations
    """
    # Get user profile
    user_profile = df_users[df_users['user_id'] == user_id].iloc[0]
    
    # Get user's historical average
    user_avg = df_transactions[
        (df_transactions['user_id'] == user_id) & 
        (df_transactions['transaction_type'] == 'debit')
    ]['amount'].mean()
    
    # Encode inputs
    user_enc = le_user.transform([user_id])[0]
    event_type_enc = le_event_type.transform([event_type])[0]
    personality_enc = le_personality.transform([user_profile['spending_personality']])[0]
    risk_enc = le_risk.transform([user_profile['risk_tolerance']])[0]
    
    # Calculate features
    quarter = (month - 1) // 3 + 1
    is_festive = 1 if month in [10, 11, 12, 1] else 0
    is_summer = 1 if month in [5, 6] else 0
    day_of_week = 3  # Assume mid-week
    is_weekend = 0
    is_recurring = 0  # Non-recurring by default
    
    # MODEL 1: Predict actual expense
    features_event = pd.DataFrame([[
        user_enc, event_type_enc, year, month, quarter,
        day_of_week, is_weekend, is_festive, is_summer,
        predicted_expense, user_profile['monthly_salary'], user_profile['savings_rate'],
        personality_enc, risk_enc, user_avg, is_recurring
    ]], columns=feature_cols_model1)
    
    actual_expense_predicted = best_model_event.predict(features_event)[0]
    actual_expense_predicted = max(actual_expense_predicted, 0)
    
    # MODEL 2: Recommend savings
    expense_to_income = actual_expense_predicted / user_profile['monthly_salary']
    expense_vs_predicted = actual_expense_predicted / (predicted_expense + 1)
    
    features_savings = pd.DataFrame([[
        user_enc, event_type_enc, year, month,
        predicted_expense, actual_expense_predicted, 
        user_profile['monthly_salary'], user_profile['savings_rate'],
        personality_enc, risk_enc, is_festive, is_summer,
        expense_to_income, expense_vs_predicted
    ]], columns=feature_cols_model2)
    
    monthly_savings_needed = best_model_savings.predict(features_savings)[0]
    monthly_savings_needed = max(monthly_savings_needed, 0)
    
    # Calculate additional insights
    budget_gap = actual_expense_predicted - predicted_expense
    total_savings_needed = max(budget_gap, 0)
    months_to_save = 3  # Recommend 3-month preparation
    
    # Determine urgency
    if expense_to_income > 0.5:
        urgency = 'High'
        urgency_msg = "⚠️ This event may significantly impact your budget!"
    elif expense_to_income > 0.3:
        urgency = 'Medium'
        urgency_msg = "⚡ Moderate financial planning needed"
    else:
        urgency = 'Low'
        urgency_msg = "✅ Manageable within your budget"
    
    # Generate recommendations
    recommendations = []
    
    if budget_gap > 0:
        recommendations.append(f"Start saving ₹{monthly_savings_needed:.0f} per month")
        recommendations.append(f"Reduce discretionary spending by ₹{(budget_gap/3):.0f}/month")
        
        # Category-specific advice
        if event_type == 'Travel':
            recommendations.append("Consider budget-friendly accommodation options")
            recommendations.append("Book tickets in advance for better deals")
        elif event_type == 'Celebration':
            recommendations.append("Plan a budget for gifts and dining")
            recommendations.append("Share costs with other participants")
        elif event_type == 'Shopping':
            recommendations.append("Create a priority list before shopping")
            recommendations.append("Look for discounts and festive offers")
    else:
        recommendations.append("Your budget estimate is realistic!")
        recommendations.append("Keep some buffer for unexpected costs")
    
    # Personality-based advice
    if user_profile['spending_personality'] == 'impulsive':
        recommendations.append("⚠️ Avoid impulse purchases during this event")
    elif user_profile['spending_personality'] == 'frugal':
        recommendations.append("💡 You're naturally good at saving - stay on track!")
    
    return {
        'user_id': user_id,
        'event_name': event_name,
        'event_type': event_type,
        'year': year,
        'month': month,
        'user_estimated_expense': predicted_expense,
        'ai_predicted_actual_expense': round(actual_expense_predicted, 2),
        'budget_gap': round(budget_gap, 2),
        'urgency': urgency,
        'urgency_message': urgency_msg,
        'total_savings_needed': round(total_savings_needed, 2),
        'recommended_monthly_saving': round(monthly_savings_needed, 2),
        'months_to_prepare': months_to_save,
        'expense_to_income_ratio': round(expense_to_income * 100, 1),
        'recommendations': recommendations
    }

# Example Usage
print("\n📊 EXAMPLE PREDICTIONS & RECOMMENDATIONS\n")

test_events = [
    ('U001', 'Trip to Goa', 'Travel', 2025, 10, 12000),
    ('U002', 'Wedding Ceremony', 'Celebration', 2025, 11, 20000),
    ('U003', 'New Laptop', 'Shopping', 2025, 8, 50000),
    ('U004', 'Family Vacation', 'Travel', 2025, 6, 30000),
    ('U005', 'Medical Checkup', 'Healthcare', 2025, 3, 5000),
]

for user, event, etype, yr, mon, est in test_events:
    result = predict_and_recommend(user, event, etype, yr, mon, est)
    
    print("="*70)
    print(f"Event: {result['event_name']} ({result['event_type']})")
    print(f"User: {result['user_id']} | Date: {result['month']}/{result['year']}")
    print("-"*70)
    print(f"📝 Your Estimate:        ₹{result['user_estimated_expense']:>10,.0f}")
    print(f"🤖 AI Predicted Actual:  ₹{result['ai_predicted_actual_expense']:>10,.2f}")
    print(f"💰 Budget Gap:           ₹{result['budget_gap']:>10,.2f}")
    print(f"⚠️  Urgency:             {result['urgency']} - {result['urgency_message']}")
    print(f"💵 Expense/Income:       {result['expense_to_income_ratio']:>10.1f}%")
    print("-"*70)
    print(f"💡 Savings Plan:")
    print(f"   • Total Savings Needed:    ₹{result['total_savings_needed']:,.2f}")
    print(f"   • Monthly Savings Target:  ₹{result['recommended_monthly_saving']:,.2f}")
    print(f"   • Preparation Time:        {result['months_to_prepare']} months")
    print(f"\n📌 Recommendations:")
    for i, rec in enumerate(result['recommendations'], 1):
        print(f"   {i}. {rec}")
    print()

# Final Summary
print("\n" + "="*70)
print("✅ SYSTEM DEPLOYMENT READY")
print("="*70)
print(f"\n🎯 Model 1 (Event Expense Predictor):")
print(f"   • Best Model: {best_model_name_event}")
print(f"   • Accuracy: {results_event[best_model_name_event]['accuracy']:.2f}%")
print(f"   • MAE: ₹{results_event[best_model_name_event]['mae']:.2f}")

print(f"\n💰 Model 2 (Savings Recommender):")
print(f"   • Best Model: {best_model_name_savings}")
print(f"   • MAE: ₹{results_savings[best_model_name_savings]['mae']:.2f}")
print(f"   • RMSE: ₹{results_savings[best_model_name_savings]['rmse']:.2f}")

print(f"\n🔧 Usage:")
print(f"   result = predict_and_recommend(")
print(f"       user_id='U001',")
print(f"       event_name='Trip to Goa',")
print(f"       event_type='Travel',")
print(f"       year=2025,")
print(f"       month=10,")
print(f"       predicted_expense=15000")
print(f"   )")

print("\n" + "="*70)

FINSIGHT: CALENDAR EVENT EXPENSE PREDICTION & SAVINGS RECOMMENDATION

MODEL 1: EVENT EXPENSE PREDICTOR

📂 Loading Datasets...
✅ Calendar events: 1768
✅ Transactions: 15883
✅ Users: 5

🔧 Engineering features for event expense prediction...
   • Calculating actual expenses for events...
   • Adding historical spending patterns...

✅ Feature matrix shape: (1768, 16)
✅ Target (actual expense) shape: (1768,)
✅ Training samples: 1414
✅ Test samples: 354

🤖 Training models for event expense prediction...

XGBoost:
   MAE: ₹9283.85 | RMSE: ₹12095.32 | R²: 0.0575 | Acc: 16.95%

LightGBM:
   MAE: ₹9068.07 | RMSE: ₹11663.84 | R²: 0.1235 | Acc: 14.97%

GradientBoosting:
   MAE: ₹9238.88 | RMSE: ₹12229.01 | R²: 0.0365 | Acc: 15.82%

RandomForest:
   MAE: ₹8806.40 | RMSE: ₹11366.18 | R²: 0.1677 | Acc: 15.82%

🏆 Best Model: XGBoost
   Accuracy: 16.95%
   MAE: ₹9283.85

MODEL 2: SAVINGS RECOMMENDATION SYSTEM

🔧 Engineering features for savings recommendation...

✅ Feature matrix shape: (1768, 14)
✅ Ta